In [3]:
import requests
from bs4 import BeautifulSoup # pip install beautifulsoup4
import pandas as pd
import requests
import re

In [32]:
def get_sgp_office_data(start, end):
    sgp_office_data = []
    for offset in range(start, end):
        url = f'https://www.99.co/singapore/commercial/rent/offices?listing_type=rent&main_category=office&map_bounds=1.5827095153768858%2C103.49449749970108%2C1.1090706240313446%2C104.12483807587296&page_num={offset}&page_size=35&path=%2Fsingapore%2Fcommercial%2Frent%2Foffices&property_segments=commercial&query_coords=1.3039947%2C103.8298507&query_limit=radius&query_type=city&radius_max=1000&rental_type=unit&show_cluster_preview=true&show_description=true&show_internal_linking=true&show_meta_description=true&show_nearby=true&zoom=15'
        html = requests.get(url, headers={'User-Agent': "Mozilla/5.0"})
        soup = BeautifulSoup(html.content, "html.parser")
        
        tags_name = soup.find_all("a", class_="_3Ajbv _30I97 _1vzK2", href=True)
        tags_district = soup.find_all("li", class_="_3WG9R", href=False, attrs={"itemprop": "addressRegion"})
        tags_address = soup.find_all("li", class_="_3WG9R _3L5OV", attrs={"itemprop": "streetAddress"})
        tags_price = soup.find_all("li", class_="JlU_W")
        tags_size = soup.find_all("li", class_="_1x-U1")
        tags_link = soup.find_all("a", href=True)

        # Extract the text values from the tags
        names = [i.text.replace("Office in ", "").strip() for i in tags_name]
        districts = [i.text.strip() for i in tags_district]
        addresses = [i.text.strip() for i in tags_address]
        prices = [i.text.strip().replace(",", "").replace("[", "").replace("]", "").replace("$", "").split("/")[0] for i in tags_price]
        sizes = [i.text.strip().replace(",", "").replace("[", "").replace("]", "").split("/")[0].split(" ")[0] for i in tags_size]
        links = [i['href'] for i in tags_link if 'enquiry_source' in i['href']]
        # Zip the values together
        rows = list(zip(names, districts, addresses, prices, sizes, links))
        sgp_office_data.extend(rows)
        
    df_sgp_office = pd.DataFrame(sgp_office_data, columns=['Name', 'District', 'Address', 'Price ($) p/m', 'Size (m2)', 'Link'])
    df_sgp_office['Price per m2'] =  (df_sgp_office['Price ($) p/m'].astype(float) / df_sgp_office['Size (m2)'].astype(float)).round(2)
    df_sgp_office['Link'] = df_sgp_office['Link'].apply(lambda x: 'https://www.99.co' + x)
    df_sgp_office = df_sgp_office.reindex(columns=[i for i in df_sgp_office.columns if i != 'Link'] + ['Link'])

    return df_sgp_office


In [50]:
df_sgp_office = get_sgp_office_data(1, 30)

In [52]:
df_sgp_office

,Name,District,Address,Price ($) p/m,Size (m2),Price per m2,Link
0,Old Birdcage Walk,D28,Old Birdcage Walk,2800,720,3.89,https://www.99.co/singapore/commercial/rent/pr...
1,Woods Square,D25,Woodlands Square,3088,560,5.51,https://www.99.co/singapore/commercial/rent/pr...
6,Mountbatten Square,D14,Mountbatten Rd,6538,1453,4.50,https://www.99.co/singapore/commercial/rent/pr...
7,Vision Exchange,D22,Venture Dr,7769,1689,4.60,https://www.99.co/singapore/commercial/rent/pr...
8,Plus,D1,Cecil St,8646,786,11.00,https://www.99.co/singapore/commercial/rent/pr...
...,...,...,...,...,...,...,...
990,CPF Tampines Building,D18,Tampines Ctrl 5,5720,1060,5.40,https://www.99.co/singapore/commercial/rent/pr...
995,Woodlands Industrial Xchange,D25,Woodlands Ave 10,1700,1000,1.70,https://www.99.co/singapore/commercial/rent/pr...
1003,Soho Life,D15,Joo Chiat Rd,3000,409,7.33,https://www.99.co/singapore/commercial/rent/pr...
1005,MYP Plaza,D1,Cecil St,13788,2293,6.01,https://www.99.co/singapore/commercial/rent/pr...


In [51]:
df_sgp_office.drop_duplicates(subset = ["Name", "Address"], inplace = True)

In [46]:
df_sgp_office

In [53]:
df_sgp_office['Longitude'] = ''
df_sgp_office['Latitude'] = ''

In [6]:
df_sgp_office = df_sgp_office.drop(index=244)
df_sgp_office = df_sgp_office.drop(index=38)

In [54]:
df_sgp_office.to_csv(r'./data/office_sgp_all.csv')

In [4]:
#df_d7 = df_sgp_office[df_sgp_office['District'] == 'D7']
df_d7 = pd.read_csv(r'../project-III-geospatial-data/data/office_sgp_coordinates.csv')

In [5]:


def add_coordinates(df):
    for index, row in df.iterrows():
        try:
            address = row['Address']
            name = row["Name"]
            url = f'https://geocode.xyz/{name} {address} Singapore?json=1'
            response = requests.get(url)
            if response.status_code == 200:
                data = response.json()
                latitude = float(data['latt'])
                longitude = float(data['longt'])
                row['Latitude'] = latitude
                row['Longitude'] = longitude
            else:
                row["No match"](f"No match found for address: {address}")
        except:
            print(f"Error getting coordinates for address: {address}")
    return df


In [10]:
df_d7

,Name,District,Address,Price ($) p/m,Size (m2),Price per m2,Link
60,Sunshine Plaza,D7,Bencoolen St,4000,517,7.74,https://www.99.co/singapore/commercial/rent/pr...
64,GR.ID,D7,Selegie Rd,128000,15000,8.53,https://www.99.co/singapore/commercial/rent/pr...
77,Jun Xin Building,D7,Nth Bridge Rd,15870,3170,5.01,https://www.99.co/singapore/commercial/rent/pr...
86,North Bridge Centre,D7,Nth Bridge Rd,1746,291,6.00,https://www.99.co/singapore/commercial/rent/pr...
87,Icb Enterprise House,D7,Middle Rd,1200,250,4.80,https://www.99.co/singapore/commercial/rent/pr...
...,...,...,...,...,...,...,...
943,Golden Mile Tower,D7,Beach Rd,2200,625,3.52,https://www.99.co/singapore/commercial/rent/pr...
944,Golden Mile Tower,D7,Beach Rd,2200,625,3.52,https://www.99.co/singapore/commercial/rent/pr...
945,Golden Mile Tower,D7,Beach Rd,2200,625,3.52,https://www.99.co/singapore/commercial/rent/pr...
974,The Plaza,D7,Beach Rd,18000,3000,6.00,https://www.99.co/singapore/commercial/rent/pr...


In [32]:
df_d7['Longitude'] = ''
df_d7['Latitude'] = ''

C:\Users\Tim_K\AppData\Local\Temp\ipykernel_8104\770581426.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_d7['Longitude'] = ''
C:\Users\Tim_K\AppData\Local\Temp\ipykernel_8104\770581426.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_d7['Latitude'] = ''


In [122]:

def add_coordinates_to_dataframe(df):
    for index, row in df.iterrows():
        address = row['Address']
        name = row['Name']
        url = f'https://geocode.xyz/{address} Singapore?json=1'
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            try:
                latitude = float(data['latt'])
                longitude = float(data['longt'])
                df.loc[index, 'Latitude'] = latitude
                df.loc[index, 'Longitude'] = longitude
            except KeyError:
                print(f"No match for {address}")
        else:
            print(f"Error getting coordinates for {address}")


,Name,District,Address,Price ($) p/m,Size (m2),Price per m2,Link,Longitude,Latitude
0,Old Birdcage Walk,D28,Old Birdcage Walk,2800,720,3.89,https://www.99.co/singapore/commercial/rent/pr...,,
1,Woods Square,D25,Woodlands Square,3088,560,5.51,https://www.99.co/singapore/commercial/rent/pr...,,
2,Woods Square,D25,Woodlands Square,9988,1722,5.80,https://www.99.co/singapore/commercial/rent/pr...,,
3,Old Birdcage Walk,D28,Old Birdcage Walk,25000,6970,3.59,https://www.99.co/singapore/commercial/rent/pr...,,
4,Old Birdcage Walk,D28,Old Birdcage Walk,4500,1030,4.37,https://www.99.co/singapore/commercial/rent/pr...,,
...,...,...,...,...,...,...,...,...,...
1010,RB Capital Building,D1,Malacca St,8000,1200,6.67,https://www.99.co/singapore/commercial/rent/pr...,,
1011,Vision Exchange,D22,Venture Dr,2800,517,5.42,https://www.99.co/singapore/commercial/rent/pr...,,
1012,The Plaza,D7,Beach Rd,18000,3000,6.00,https://www.99.co/singapore/commercial/rent/pr...,,
1013,Peninsula Plaza,D6,Nth Bridge Rd,4800,1000,4.80,https://www.99.co/singapore/commercial/rent/pr...,,


In [132]:
add_coordinates_to_dataframe(df_sgp_office)


Error getting coordinates for Mountbatten Rd
Error getting coordinates for Cecil St
Error getting coordinates for Keppel Rd
Error getting coordinates for Sin Ming Lane
Error getting coordinates for Orchard Rd
Error getting coordinates for Kaki Bt Pl
Error getting coordinates for Upp Serangoon Rd
Error getting coordinates for Selegie Rd
Error getting coordinates for Temasek Boulevard
Error getting coordinates for Cecil St
Error getting coordinates for Nth Bridge Rd
Error getting coordinates for Battery Rd
Error getting coordinates for Eu Tong Sen St
Error getting coordinates for Jln Pemimpin
Error getting coordinates for Havelock Rd
Error getting coordinates for New Bridge Rd
Error getting coordinates for Orchard Rd
Error getting coordinates for Serangoon Rd
Error getting coordinates for Beach Rd
Error getting coordinates for Mosque St
Error getting coordinates for Prinsep St
Error getting coordinates for Toh Guan Rd East
Error getting coordinates for Science Centre Rd
Error getting coo

In [130]:
df_sgp_office['Latitude'].unique()


array([1.40991, 1.43601, 1.30468, 1.32989, 1.27945, 1.27186, 1.2849,
       1.35759, 1.29765, 1.33185, 1.30548, 1.30054, 1.29345, 1.29109,
       1.42327, 1.28592, 1.2892, 1.34868, 1.29013, 1.28088, 1.33567,
       1.31887, 1.34541, 1.30348, 1.31923, 1.28339, '', 1.33583, 1.33108,
       1.30515, 1.29823, 1.3113, 1.352, 1.27182, 1.28322, 1.27697,
       1.28311, 1.29029, 1.28107, 1.32566, 1.28755, 48.74106, 1.28726,
       1.30667, 1.28012, 1.30399, 1.29701, 1.29462, 1.34068, 1.31783,
       1.32051, 1.31391, 1.29809, 1.30044, 1.28453, 1.31588, 1.38058,
       1.2854, 1.27703, 1.43518, 1.36496, 4.4772, 1.34981, 1.29042,
       1.27909, 1.30231, 1.27879, 1.27996, 1.32242, 1.28507, 1.30138,
       1.28084, 1.27608, 1.28017, 1.29786, 1.28395, 1.34433, 1.30711,
       1.2768, 1.35265, 1.33067, 1.2835, 1.29279, 1.32959, 1.33945,
       1.28947, 1.3163, 1.29131, 1.39035, 1.33622, 1.28357, 1.33105,
       1.30963, 1.29925, 1.28274, 1.28179, 1.28796, 1.32891, 1.28233,
       1.29117, 1.27468, 

In [116]:
df_sgp_office

,Name,District,Address,Price ($) p/m,Size (m2),Price per m2,Link,Longitude,Latitude
0,Old Birdcage Walk,D28,Old Birdcage Walk,2800,720,3.89,https://www.99.co/singapore/commercial/rent/pr...,103.87024,1.40991
1,Woods Square,D25,Woodlands Square,3088,560,5.51,https://www.99.co/singapore/commercial/rent/pr...,103.78569,1.43601
6,Mountbatten Square,D14,Mountbatten Rd,6538,1453,4.50,https://www.99.co/singapore/commercial/rent/pr...,103.88345,1.30468
7,Vision Exchange,D22,Venture Dr,7769,1689,4.60,https://www.99.co/singapore/commercial/rent/pr...,103.74461,1.32989
8,Plus,D1,Cecil St,8646,786,11.00,https://www.99.co/singapore/commercial/rent/pr...,103.84836,1.27945
...,...,...,...,...,...,...,...,...,...
990,CPF Tampines Building,D18,Tampines Ctrl 5,5720,1060,5.40,https://www.99.co/singapore/commercial/rent/pr...,103.94174,1.35368
995,Woodlands Industrial Xchange,D25,Woodlands Ave 10,1700,1000,1.70,https://www.99.co/singapore/commercial/rent/pr...,103.81198,1.45701
1003,Soho Life,D15,Joo Chiat Rd,3000,409,7.33,https://www.99.co/singapore/commercial/rent/pr...,,
1005,MYP Plaza,D1,Cecil St,13788,2293,6.01,https://www.99.co/singapore/commercial/rent/pr...,103.84836,1.27945


In [125]:
df_sgp_office.to_csv('./data/office_all_sgp_coordinates.csv')

In [23]:
df_d7 = df_d7.drop_duplicates(subset = ["Name", "Address"])

In [28]:
df_d7.drop([17, 65])

,Unnamed: 0,Name,District,Address,Price ($) p/m,Size (m2),Price per m2,Link,Longitude,Latitude
0,60,Sunshine Plaza,D7,Bencoolen St,4000,517,7.74,https://www.99.co/singapore/commercial/rent/pr...,103.84965,1.29823
1,64,GR.ID,D7,Selegie Rd,128000,15000,8.53,https://www.99.co/singapore/commercial/rent/pr...,103.84925,1.30054
2,77,Jun Xin Building,D7,Nth Bridge Rd,15870,3170,5.01,https://www.99.co/singapore/commercial/rent/pr...,103.96126,1.34068
3,86,North Bridge Centre,D7,Nth Bridge Rd,1746,291,6.00,https://www.99.co/singapore/commercial/rent/pr...,103.84896,1.29042
4,87,Icb Enterprise House,D7,Middle Rd,1200,250,4.80,https://www.99.co/singapore/commercial/rent/pr...,103.70291,1.33390
5,93,Fortune Centre,D7,Middle Rd,8000,1668,4.80,https://www.99.co/singapore/commercial/rent/pr...,103.85559,1.29701
6,94,The Plaza,D7,Beach Rd,5600,1249,4.48,https://www.99.co/singapore/commercial/rent/pr...,103.86454,1.30348
7,118,Bylands Building,D7,Middle Rd,5900,1323,4.46,https://www.99.co/singapore/commercial/rent/pr...,103.85194,1.29954
8,142,Bugis Junction Towers,D7,Victoria St,94278,9924,9.50,https://www.99.co/singapore/commercial/rent/pr...,103.85122,1.29462
11,222,Chye Sing Building,D7,Beach Rd,7500,1700,4.41,https://www.99.co/singapore/commercial/rent/pr...,103.86454,1.30348


In [29]:
df_d7.to_csv('./data/office_sgp_coordinates.csv')